In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
tf.__version__

'2.7.0'

In [3]:
train_set = pd.read_csv('train.csv')
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        # BytesList won't unpack a string from an EagerTensor.
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [28]:
def encode_gender(g):
    return 0 if g == 'male' else 1

def decode_gender(g):
    return 'male' if g== 0 else 'female'

def encode_ticket(ticket):
    t = ticket.split()

    if t[-1].isalpha():
        return 0
    return int(t[-1])

def encode_cabin(cabin):
    if not cabin: # handle NaN
        return 0,0
    
    # todo : parse string part

    # todo : parse numeric part
    return 0, 0

def encode_embarked(embarked_class):
    if embarked_class == 'S': return 1
    if embarked_class == 'C': return 2
    if embarked_class == 'Q': return 3
    return 0

def serialize_titanic_data(pass_id, label, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender):
    ticket = encode_ticket(ticket)
    embarked = encode_embarked(embarked)
    gender = encode_gender(gender)
    # cabin_type, cabin_id = encode_cabin(cabin)

    X = [pass_id, label, pclass, age, ticket, fare, gender, embarked]
    y = label
    return X, y
    # ============ FOR TFRECORDS ======================
    
    # FIXME : not gonna use Name for now
    # if label:
    #     feature = {
    #         'pass_id' : _int64_feature(pass_id),
    #         'label' : _int64_feature(label),
    #         'pclass' : _int64_feature(pclass),
    #         'sibsp' : _int64_feature(sibsp),
    #         'parch' : _int64_feature(parch),
    #         'ticket' : _int64_feature(ticket),
    #         'fare' : _float_feature(fare),
    #         'embarked' : _int64_feature(embarked),
    #         'gender' : _int64_feature(gender),
    #         'cabin_type' : _int64_feature(cabin_type),
    #         'cabin_id' : _int64_feature(cabin_id),
    #     }
    # else:
    #     feature = {
    #         'pass_id' : _int64_feature(pass_id),
    #         'pclass' : _int64_feature(pclass),
    #         'sibsp' : _int64_feature(sibsp),
    #         'parch' : _int64_feature(parch),
    #         'ticket' : _int64_feature(ticket),
    #         'fare' : _float_feature(fare),
    #         'embarked' : _int64_feature(embarked),
    #         'gender' : _int64_feature(gender),
    #         'cabin_type' : _int64_feature(cabin_type),
    #         'cabin_id' : _int64_feature(cabin_id),
    #     }

    # # Create a Features message using tf.train.Example

    # example_proto = tf.train.Example(
    #     features=tf.train.Features(feature=feature)
    # )
    # return example_proto.SerializeToString()
    

In [46]:
train_set['Ticket']
import math

In [47]:
X_train, y_train =[], []
for _, row in train_set.iterrows():
    pass_id, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender = row.PassengerId, \
                row.Pclass, row.Name, row.Age, row.SibSp, row.Parch, row.Ticket, row.Fare, row.Cabin, row.Embarked, row.Sex 
    label = row.Survived
    X , y = serialize_titanic_data(pass_id, label, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender)
    for i,x in enumerate(X):
        if math.isnan(x):
            X[i] = -1
    X_train.append(X)
    y_train.append(y)
    
# print(y_train)
train_ds = tf.data.Dataset.from_tensors((X_train, y_train))
# print(y_train)

In [8]:
def writeTFRecord(filename='test.tfrecord', df=None):
    with tf.io.TFRecordWriter(filename) as writer:
        for _, row in df.iterrows():
            pass_id, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender = row.PassengerId, \
                 row.Pclass, row.Name, row.Age, row.SibSp, row.Parch, row.Ticket, row.Fare, row.Cabin, row.Embarked, row.Sex
            try:
                label = row.Survived
            except AttributeError:
                label = None

            data = serialize_titanic_data(pass_id, label, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender)
            writer.write(data)

            
        # for i in range(df):
        #     # example = serialize_example(
        #     #     feature0[i], feature1[i], feature2[i], feature3[i])
        #     # writer.write(example)
        #     pass

In [13]:
# writeTFRecord('train.tfrecord', train_set)

In [14]:
# raw_ds = tf.data.TFRecordDataset('train.tfrecord')

In [15]:
# example = None
# for raw_record in raw_ds.take(1):
#     example = tf.train.Example()
#     example.ParseFromString(raw_record.numpy())

In [16]:
# test_set = pd.read_csv('test.csv')
# writeTFRecord('test.tfrecord', test_set)

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_model():
    model = Sequential()
    model.add(Dense(320,activation='relu', input_shape=[8]))
    model.add(Dense(120, activation='relu', input_shape=[320]))
    model.add(Dense(12, activation='relu', input_shape = [120]))
    model.add(Dense(1, activation='softmax', input_shape = [1]))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.build(input_shape=[8])
    return model

In [49]:
m = create_model()

In [50]:
m.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 320)               2880      
                                                                 
 dense_9 (Dense)             (None, 120)               38520     
                                                                 
 dense_10 (Dense)            (None, 12)                1452      
                                                                 
 dense_11 (Dense)            (None, 1)                 13        
                                                                 
Total params: 42,865
Trainable params: 42,865
Non-trainable params: 0
_________________________________________________________________


In [51]:
for X, y in train_ds.take(1):
    X= X.numpy()
    y = y.numpy()
    print(X)
    print(y)

[[  1.       0.       3.     ...   7.25     0.       1.    ]
 [  2.       1.       1.     ...  71.2833   1.       2.    ]
 [  3.       1.       3.     ...   7.925    1.       1.    ]
 ...
 [889.       0.       3.     ...  23.45     1.       1.    ]
 [890.       1.       1.     ...  30.       0.       2.    ]
 [891.       0.       3.     ...   7.75     0.       3.    ]]
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 0 1 0 1 1 1 0 1

In [52]:
m.fit(train_ds, epochs = 20)

Epoch 1/20
1/1 [==============================] - 0s 211ms/step - loss: 0.6162
Epoch 2/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6162
Epoch 3/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6162
Epoch 4/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6162
Epoch 5/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6162
Epoch 6/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6162
Epoch 7/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6162
Epoch 8/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6162
Epoch 9/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6162
Epoch 10/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6162
Epoch 11/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6162
Epoch 12/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6162
Epoch 13/20
1/1 [==============================] - 0s 5ms/step - loss: 

In [53]:
print(X_train[0])
m.predict(tf.data.Dataset.from_tensors(X_train[0:5]))

[1, 0, 3, 22.0, 21171, 7.25, 0, 1]


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [54]:
test_set = pd.read_csv('test.csv')
test_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [55]:
X_test, pid = [], []
for _, row in test_set.iterrows():
    pass_id, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender = row.PassengerId, \
                row.Pclass, row.Name, row.Age, row.SibSp, row.Parch, row.Ticket, row.Fare, row.Cabin, row.Embarked, row.Sex 
    label = None
    X , y = serialize_titanic_data(pass_id, label, pclass, name, age,sibsp,parch, ticket, fare, cabin, embarked, gender)
    X_test.append(X)
    pid.append(pass_id)

    
# print(y_train)
test_ds = tf.data.Dataset.from_tensors((X_train, y_train))